In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib

def etl_pipeline(csv_file, output_file="processed_data.csv", pipeline_file="preprocessing_pipeline.joblib"):
    try:
        # 1. Extract (Load)
        df = pd.read_csv(csv_file)
        print(f"Original DataFrame shape: {df.shape}")

        # Handle missing values
        for col in df.columns:
            if pd.api.types.is_numeric_dtype(df[col]):
                df[col] = df[col].fillna(df[col].mean())  # Assign back to the column
            else:
                df[col] = df[col].fillna(df[col].mode()[0])  # Assign back to the column

        numerical_features = df.select_dtypes(include=['number']).columns.tolist()
        categorical_features = df.select_dtypes(include=['object']).columns.tolist()

        numerical_transformer = Pipeline(steps=[('scaler', StandardScaler())])
        categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

        preprocessor = ColumnTransformer(
            transformers=[
                ('num', numerical_transformer, numerical_features),
                ('cat', categorical_transformer, categorical_features)
            ])
        
        pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

        # Fit and transform the data
        processed_data = pipeline.fit_transform(df)
        print(f"Processed data shape: {processed_data.shape}")

        # Get feature names after one-hot encoding
        numerical_columns = numerical_features
        categorical_columns = pipeline.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_features).tolist()
        
        print(f"Number of numerical columns: {len(numerical_columns)}")
        print(f"Number of categorical columns after one-hot encoding: {len(categorical_columns)}")
        print(f"Total number of columns expected: {len(numerical_columns) + len(categorical_columns)}")

        final_columns = numerical_columns + categorical_columns

        # Convert back to DataFrame
        processed_df = pd.DataFrame(processed_data, columns=final_columns)

        # 3. Load
        processed_df.to_csv(output_file, index=False)
        print(f"Processed data saved to {output_file}")
        joblib.dump(pipeline, pipeline_file)
        print(f"Preprocessing pipeline saved to {pipeline_file}")

    except FileNotFoundError:
        print(f"Error: File '{csv_file}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    etl_pipeline(r"C:\Users\saisaran.tummala/railwayCSV.csv")  

Original DataFrame shape: (11113, 5)
Processed data shape: (11113, 9440)
Number of numerical columns: 1
Number of categorical columns after one-hot encoding: 9439
Total number of columns expected: 9440
An error occurred: Shape of passed values is (11113, 1), indices imply (11113, 9440)
